In [1]:
%run Package.ipynb

In [2]:
path = 'E:/MACHINE LEARNING/3.Application CC/Data_ML'

In [3]:
feature_final = ['VAR1', 'GENDER', 'VAR2', 'VAR3', 'VAR5', 'VAR4', 'VAR8', 'VAR15']

model_final = lgb.Booster(model_file='Model parameters.txt')

# Sample

In [ ]:
import pyodbc
import pandas as pd
conn=pyodbc.connect('DSN=Modelling2;'
                    'Trusted_Connection=yes;')
monitor_202208=pd.read_sql("SELECT * FROM MODELLING2.dbo.MONITORING_202208 WHERE FINAL_STATUS = 'APPROVED'",conn)
monitor_202208.columns = [col.upper() for col in monitor_202208.columns]

monitor_202208.to_pickle(path+'/d_monitor_202208.pkl')

# Chạy nếu insert thêm tháng

In [4]:
monitor_202208 = pd.read_pickle(path+'/d_monitor_202208.pkl')

In [5]:
monitor = pd.read_pickle(path+'/d_monitor_update.pkl')

In [6]:
monitor = monitor[['APP_ID_C','YEARMONTH']+feature_final].append(monitor_202208[['APP_ID_C','YEARMONTH']+feature_final])

In [7]:
monitor.groupby('YEARMONTH')['YEARMONTH'].agg(Observations=('count'))

Observations
YEARMONTH              
202110             6429
202111             8116
202112             9520
202201             6658
202202             4348
202203             8522
202204             7960
202205             8728
202206            10126
202207            12155
202208            18503

### Transform

In [8]:
GRP_GENDER={'F': 2.23487, 'M': 5.45343}

In [9]:
# ch_varlist = ['GENDER','HOME_PROPERTY']
ch_varlist = ['GENDER']

for var in ch_varlist:
    
    monitor['GRP_'+var] = monitor[var].map(globals()['GRP_'+var]).astype('float')
    monitor = monitor.rename(columns={var:'ORG_'+var, 'GRP_'+var:var})

    
monitor.shape

(101065, 11)

In [10]:
monitoring['CASHIN_AMT_3M'] = np.where(monitoring['CASHIN_AMT_3M']<=0, monitoring['CASHIN_AMT_3M'],
                                   np.where(monitoring['CASHIN_AMT_3M']<=37000000, 37000000,
                                            np.where(monitoring['CASHIN_AMT_3M']>37000000, 37500000,
                                                     monitoring['CASHIN_AMT_3M'])))


monitoring['VAR10'] = np.where(monitoring['VAR10']<=0, 0,
                                  np.where(monitoring['VAR10']<=1300000,monitoring['VAR10'],
                                           np.where(monitoring['VAR10']<=3150000,3150000,
                                                    monitoring['VAR10'])))


In [15]:
train=pd.read_pickle(path+'/d_train_approved_model.pkl')
valid=pd.read_pickle(path+'/d_valid_approved_model.pkl')
oot=pd.read_pickle(path+'/d_oot_approved_model.pkl')
    
train.columns = [col.upper() for col in train.columns]
valid.columns = [col.upper() for col in valid.columns]
oot.columns = [col.upper() for col in oot.columns]

train.shape, valid.shape, oot.shape

((21505, 233), (9216, 233), (14545, 218))

## Update value missing in scoring data

In [13]:
nan_cols = [i for i in train[feature_final].columns if train[i].isnull().sum()>0]
nan_cols

[]

In [14]:
set([i for i in monitor[feature_final].columns if monitor[i].isnull().sum()>0]) - set(nan_cols)

set()

In [15]:
monitor.to_pickle(path+'/monitor_all.pkl')

# Không có tháng mới thì chạy từ đây

In [13]:
train=pd.read_pickle(path+'/d_train_approved_model.pkl')
valid=pd.read_pickle(path+'/d_valid_approved_model.pkl')
oot=pd.read_pickle(path+'/d_oot_approved_model.pkl')
    
train.columns = [col.upper() for col in train.columns]
valid.columns = [col.upper() for col in valid.columns]
oot.columns = [col.upper() for col in oot.columns]

train.shape, valid.shape, oot.shape

((21505, 233), (9216, 233), (14545, 218))

In [15]:
y_train_pred = model_final.predict(train[feature_final], num_iteration=model_final.best_iteration)
y_valid_pred = model_final.predict(valid[feature_final], num_iteration=model_final.best_iteration)
y_oot_pred = model_final.predict(oot[feature_final], num_iteration=model_final.best_iteration)
train['Score'] = y_train_pred
train['Score'] = round(train['Score']*1000,0)

valid['Score'] = y_valid_pred
valid['Score'] = round(valid['Score']*1000,0)

oot['Score'] = y_oot_pred
oot['Score'] = round(oot['Score']*1000,0)

In [4]:
monitor = pd.read_pickle(path+'/monitor_all.pkl')
monitor.columns = [col.upper() for col in monitor.columns]

monitor.shape

(101065, 11)

In [ ]:
monitor.head(3)

In [ ]:
monitor[feature_final].isnull().sum()

# Scoring

In [7]:
y_monitor = model_final.predict(monitor[feature_final], num_iteration=model_final.best_iteration)
monitor['Score'] = y_monitor
monitor['Score'] = round(monitor['Score']*1000,0)

In [8]:
bin = [-np.inf, 773, 900, 927, 950, 960, 975, 982, 990, 995, np.inf]

df =monitor[monitor['YEARMONTH']>=202110][monitor['YEARMONTH']<=202208].copy()
df=df.assign(Band=lambda x: pd.cut(x['Score'],bins=bin,
                                       right=True))

In [9]:
df.groupby('YEARMONTH')['Score'].agg(Observations=('count'))

Observations
YEARMONTH              
202110             6429
202111             8116
202112             9520
202201             6658
202202             4348
202203             8522
202204             7960
202205             8728
202206            10126
202207            12155
202208            18503

In [10]:
df['Score'].isnull().sum(), df['YEARMONTH'].isnull().sum()

(0, 0)

In [ ]:
df.head(2)

# PSI

In [ ]:
bin = [-np.inf, 773, 900, 927, 950, 960, 975, 982, 990, 995, np.inf]

model_stab(train=train['Score'], outtime=df['Score'], sample=df['YEARMONTH'], bins=bin)